# Todos

## General

- bale ta ballast trips apo ships tensor pisw sto ships df ❌
- ananewne ka8w fora ta contracts_df kai ships_df me oti allagh ginetai stous contracts_tensor kai ships_tensor


## Step

### Allakse to state otan allazei h mera ❌

afou looparw se ola ta available ships shmainei oti h mera allazei

   #### - diamorfwse katallhla to ships log ❌
   
          * to ships log prepei na exei values >= 0 ❌
          * apo mh mhdenika ships log values afairese 1 epeidh perase mia mera ❌
          * an prokypsoun ships_log[ship] = 0 tote kane to `ship_availability` autou tou ship 1 ❌
          * osa ships einai available (exoun ships_log[ship] == 0) bainoun se lista `available_ships` pou 8a thn allazeis ka8e mera. ❌
           

   #### -  kane generate 4 nea contracts ❌

### Pws ypologizw to `reward` apo to action  ✅
    

   #### - Value of contract ✅
   #### - CII ✅
   #### - Lateness ✅
   #### - Ftiakse sugentrwtiko `calculate_reward(selected_ship_idx,selected_contract,selected_speed)` function ✅

### Pws na kanw swsta update to `state`  ✅
  

      
   #### - contract tensor ✅
   #### - ship tensor ✅
   #### - contract mask ✅
   #### - ship mask ✅
   #### - ship_log ✅
        

## Train 

ti einai ta `advantages`, `returns` kai giati xrhsimopoioyntai:
        
- ❌ sthn baseline_net.update(observations, returns) gia to update to baselineNet
- ❌ sthn PolicyGradient.update_policy(observations, actions, advantages)gia to update ths policygradient 

# Code

In [1]:
# autoreloading modules that i change in vscode
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [169]:
# run this cell when needing to reload the classes CarbonEnv, BaselineNet, PolicyNet

del sys.modules['models.models']
del sys.modules['env.env']
del sys.modules['training.training_functions']
del sys.modules['utils.utils']

from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from training.training_functions import PolicyGradient

In [170]:
from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from training.training_functions import PolicyGradient

In [ ]:
envo = CarbonEnv()

In [5]:
model = PolicyGradient(envo)

In [6]:
year_data, num_steps = model.train()

Ksekina to year: 0
Xronia: 1 kai mera: 0
Ta available ships einai [1, 2, 3, 4]
Phra contract gia to available ship 1
Den phra contract gia to available ship 2 
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 1 kai mera: 1
Ta available ships einai [2, 3, 4]
Phra contract gia to available ship 2
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 1 kai mera: 2
Ta available ships einai [2, 3, 4]
Phra contract gia to available ship 2
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 1 kai mera: 3
Ta available ships einai [2, 3, 4]
Phra contract gia to available ship 2
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 1 kai mera: 4
Ta available ships einai [2, 3, 4]
Phra contract gia to available ship 2
Den phra contract gia to available ship 3 
Den phra contract gia to available ship 4 
Xronia: 0, sunolo apo steps: 1

In [171]:
baseline_net = BaselineNet(128,1)
policy_net = PolicyNet(128,13)

In [8]:
rewards_array = year_data['reward']

In [9]:
states_array = year_data['states']

In [10]:
actions_array = year_data['action']

In [27]:
list_of_states = [list(state.values()) for state in states_array ]

In [34]:
a = list_of_states[0]

In [35]:
b = list_of_states[1]

In [44]:
list_of_states_0 = tf.stack([state[0] for state in list_of_states],0)

In [45]:
list_of_states_1 = tf.stack([state[1] for state in list_of_states],0)

In [49]:
dict_of_states = {"contracts_state":list_of_states_0,
                  "ships_state":list_of_states_1}

In [172]:
def get_returns(rewards_array):
        T = len(rewards_array)
        discounts = np.logspace(0, T, num=T, base=0.99, endpoint=False)
        returns = np.array([np.sum(discounts[: T - t] * rewards_array[t:]) for t in range(T)])
        return returns

In [173]:
returns = get_returns(rewards_array)

In [155]:
returns

array([-10100.11362652, -10224.55359376,  -9317.73090279,  -8401.74838665,
        -7476.51352187,  -7567.91914613,  -6634.26176377,  -5691.17349876,
        -5764.46315942,  -4812.58904992,  -3851.10005042,  -3905.72088461,
        -2935.07160061,  -1954.6177784 ,  -1990.        ,  -1000.        ])

In [174]:
baseline_net.forward(states_array[0])

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-355.66592], dtype=float32)>

In [177]:
def get_advantage(returns, states):
    value_array = np.array([])
    for state in states:
        value = baseline_net.forward(state).numpy()
        value_array = np.append(value_array,value)
    advantages = returns - value_array
    advantages = (advantages - np.mean(advantages)) / np.sqrt(np.sum(advantages ** 2))
    return advantages

In [178]:
advantages = get_advantage(returns,states_array)

In [179]:
advantages

array([-0.1794018 , -0.18450652, -0.14730726, -0.10973225, -0.07177956,
       -0.07552916, -0.0372291 ,  0.00145785, -0.0015486 ,  0.03749874,
        0.0769405 ,  0.07469987,  0.1145174 ,  0.15473714,  0.15328571,
        0.19389703])

In [185]:
returns

array([-10100.11362652, -10224.55359376,  -9317.73090279,  -8401.74838665,
        -7476.51352187,  -7567.91914613,  -6634.26176377,  -5691.17349876,
        -5764.46315942,  -4812.58904992,  -3851.10005042,  -3905.72088461,
        -2935.07160061,  -1954.6177784 ,  -1990.        ,  -1000.        ])

In [219]:
preds = tf.concat([baseline_net.model(state) for state in states_array],axis=0)
preds

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([-355.66592, -355.66592, -355.66592, -355.66592, -355.6204 ,
       -355.6204 , -355.6204 , -355.62103, -355.62103, -355.62103,
       -355.62134, -355.62134, -355.62134, -355.6219 , -355.6219 ,
       -355.6219 ], dtype=float32)>

In [223]:
tf.keras.losses.mean_squared_error(y_true=returns,y_pred=preds)

<tf.Tensor: shape=(), dtype=float32, numpy=37141220.0>

In [222]:
returns

array([-10100.11362652, -10224.55359376,  -9317.73090279,  -8401.74838665,
        -7476.51352187,  -7567.91914613,  -6634.26176377,  -5691.17349876,
        -5764.46315942,  -4812.58904992,  -3851.10005042,  -3905.72088461,
        -2935.07160061,  -1954.6177784 ,  -1990.        ,  -1000.        ])

In [224]:
states_array

[{'contracts_state': <tf.Tensor: shape=(4, 10), dtype=float32, numpy=
  array([[3.0000e+00, 6.0000e+00, 1.0000e+00, 0.0000e+00, 7.0000e+00,
          4.3747e+04, 8.0000e+00, 1.0000e+00, 8.4600e+02, 5.0000e+00],
         [9.0000e+00, 2.0000e+00, 3.0000e+00, 0.0000e+00, 1.5000e+01,
          6.1231e+04, 1.6000e+01, 0.0000e+00, 2.7640e+03, 1.1000e+01],
         [7.0000e+00, 5.0000e+00, 4.0000e+00, 0.0000e+00, 8.0000e+00,
          7.5258e+04, 9.0000e+00, 1.0000e+00, 1.2230e+03, 1.0000e+01],
         [2.0000e+00, 3.0000e+00, 3.0000e+00, 0.0000e+00, 1.0000e+01,
          6.4033e+04, 1.1000e+01, 1.0000e+00, 1.6530e+03, 1.0000e+01]],
        dtype=float32)>,
  'ships_state': <tf.Tensor: shape=(4, 11), dtype=float32, numpy=
  array([[1.0000000e+00, 5.0000000e+04, 5.3851833e+00, 1.9075143e-01,
          2.0000000e+00, 1.4000000e+01, 0.0000000e+00, 5.8500000e+02,
          6.9600000e+02, 4.9200000e+02, 2.2370000e+03],
         [2.0000000e+00, 6.0000000e+04, 4.8078370e+00, 0.0000000e+00,
        

In [225]:
actions_array

[5, 12, 12, 12, 5, 12, 12, 5, 12, 12, 5, 12, 12, 5, 12, 12]

In [226]:
advantages

array([-0.1794018 , -0.18450652, -0.14730726, -0.10973225, -0.07177956,
       -0.07552916, -0.0372291 ,  0.00145785, -0.0015486 ,  0.03749874,
        0.0769405 ,  0.07469987,  0.1145174 ,  0.15473714,  0.15328571,
        0.19389703])

In [229]:
log_probs = tf.concat([self.policy_net.action_distribution(state)[1].log_prob(action) for state, action in zip(states_array,actions_array)],axis=0)

In [239]:
actions_array[0]

5

In [244]:
policy_net.action_distribution(states_array[0])[1].log_prob(actions_array[0])

<tf.Tensor: shape=(), dtype=float32, numpy=-inf>

In [238]:
log_probs

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([      -inf, -3515.7136, -3515.7136, -3515.7136,       -inf,
       -3515.673 , -3515.673 ,       -inf, -3515.6736, -3515.6736,
             -inf, -3515.6743, -3515.6743,       -inf, -3515.675 ,
       -3515.675 ], dtype=float32)>

In [254]:
minimum_val = tf.reduce_min(tf.boolean_mask(log_probs, tf.math.is_finite(log_probs)))

In [258]:
log_probs_no_inf = tf.where(tf.math.is_inf(log_probs), 1000 * minimum_val, log_probs)

In [259]:
entropies = tf.concat([policy_net.action_distribution(state)[1].entropy() for state in states_array],axis=0)

In [260]:
-tf.math.reduce_mean(log_probs_no_inf * tf.cast(advantages, tf.float32) + 0.001 * entropies, )

<tf.Tensor: shape=(), dtype=float32, numpy=-3961.2852>

In [ ]:
def update_policy(states_array, one_action, one_advantage):
    #one_state is already a tensor
    one_action = tf.convert_to_tensor(one_action)
    one_advantage = tf.convert_to_tensor(one_advantage)
    with tf.GradientTape() as tape:
        log_prob = policy_net.action_distribution(one_state)[1].log_prob(one_action)
        loss = -tf.math.reduce_mean(log_prob * tf.cast(one_advantage, tf.float32))
    grads = tape.gradient(loss, policy_net.policy_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, policy_net.policy_model.trainable_weights))

In [ ]:
each_year_actions_list = np.load('results_all/results_3/actions.npy',allow_pickle=True)

In [ ]:
each_year_baseline_loss_list = np.load('results_all/results_3/baseline_loss.npy',allow_pickle=True)

In [ ]:
each_year_policynet_loss_list = np.load('results_all/results_3/policynet_loss.npy',allow_pickle=True)

In [ ]:
each_year_avg_reward_list = np.load('results_all/results_3/avg_rewards.npy',allow_pickle=True)

In [ ]:
each_year_reward_list = np.load('results_all/results_3/rewards.npy',allow_pickle=True)

In [ ]:
print(each_year_actions_list[0])

### Tensorflow Tip

Assign values to a tensor at specific indices

In [ ]:
input_array = np.array([2, 4, 7, 11, 3, 8, 9, 19, 11, 7])
inplace_array = np.array([10, 20])
indices_array = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

# [[2], [6]] 
indices = tf.cast(tf.where(tf.equal(indices_array,1)),tf.int32)
print(indices)

# [0, 0, 10, 0, 0, 0, 20, 0, 0, 0]
scatter = tf.scatter_nd(indices, inplace_array, shape=tf.shape(input_array))
print(scatter)

# [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
inverse_mask = tf.cast(tf.math.logical_not(indices_array), tf.int32)
print(inverse_mask)

# [2, 4, 0, 11, 3, 8, 0, 19, 11, 7]
input_array_zero_out = tf.multiply(inverse_mask, input_array)
print(input_array_zero_out)

# [2, 4, 10, 11, 3, 8, 20, 19, 11, 7]
output = tf.add(input_array_zero_out, tf.cast(scatter, tf.int32))
print(output)

In [ ]:
# [[2], [6]] 
indices = tf.cast(tf.where(tf.equal(indices_array,1)),tf.int32)
print(indices)

In [ ]:
# [0, 0, 10, 0, 0, 0, 20, 0, 0, 0]
scatter = tf.scatter_nd(indices, inplace_array, shape=tf.shape(input_array))
print(scatter)

In [ ]:
# [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
inverse_mask = tf.cast(tf.math.logical_not(indices_array), tf.int32)
print(inverse_mask)

In [ ]:
# [2, 4, 0, 11, 3, 8, 0, 19, 11, 7]
input_array_zero_out = tf.multiply(inverse_mask, input_array)
print(input_array_zero_out)

In [ ]:
# [2, 4, 10, 11, 3, 8, 20, 19, 11, 7]
output = tf.add(input_array_zero_out, tf.cast(scatter, tf.int32))
print(output)

In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type